# Test CodeAutoMergeEditBlock

This notebook tests the CodeAutoMergeEditBlock class's ability to parse different code block formats.

In [ ]:
import sys
sys.path.append("../../")

from src.autocoder.common.code_auto_merge_editblock import CodeAutoMergeEditBlock
from dataclasses import dataclass
import byzerllm

@dataclass
class MockArgs:
    file: str = "test.py"
    source_dir: str = "."
    editblock_similarity: float = 0.8
    request_id: str = None

In [ ]:
# Test data with two different formats
test_input = """
```python
##File: src/autocoder/index/index.py
<<<<<<< SEARCH
    # Print final statistics
    logger.info("\n=== Build Index and Filter Files Summary ===\n")
    return source_code
=======
    # Print final statistics
    logger.info("\n=== Build Index and Filter Files Summary ===\n")
    # Send filter end event
    if args.request_id:
        queue_communicate.send_event_no_wait(
            request_id=args.request_id,
            event=CommunicateEvent(
                event_type=CommunicateEventType.CODE_INDEX_FILTER_END.value,
                data=f"Completed filtering. Final files selected: {stats['final_files']}"
            )
        )
    return source_code
>>>>>>> REPLACE
```

```python:src/autocoder/index/index.py
<<<<<<< SEARCH
    # Print final statistics
    logger.info("\n=== Build Index and Filter Files Summary ===\n")
    return source_code
=======
    # Print final statistics
    logger.info("\n=== Build Index and Filter Files Summary ===\n")
    # Send filter end event
    if args.request_id:
        queue_communicate.send_event_no_wait(
            request_id=args.request_id,
            event=CommunicateEvent(
                event_type=CommunicateEventType.CODE_INDEX_FILTER_END.value,
                data=f"Completed filtering. Final files selected: {stats['final_files']}"
            )
        )
    return source_code
>>>>>>> REPLACE
```
"""

In [ ]:
# Initialize CodeAutoMergeEditBlock
llm = byzerllm.ByzerLLM()
args = MockArgs()
merger = CodeAutoMergeEditBlock(llm, args)

# Parse the test input
results = merger.parse_whole_text(test_input)

# Verify results
assert len(results) == 2, f"Expected 2 results, got {len(results)}"

# Both results should have the same path
assert results[0].path == "src/autocoder/index/index.py"
assert results[1].path == "src/autocoder/index/index.py"

# Both results should have the same content structure
for result in results:
    assert "<<<<<<< SEARCH" in result.content
    assert "=======" in result.content
    assert ">>>>>>> REPLACE" in result.content
    
print("All tests passed!")

In [ ]:
# Let's also test the get_edits method to ensure we can extract the search and replace blocks correctly
edits = merger.get_edits(test_input)

# We should have 2 edits
assert len(edits) == 2, f"Expected 2 edits, got {len(edits)}"

# Each edit should be a tuple of (path, head, update)
for path, head, update in edits:
    assert path == "src/autocoder/index/index.py"
    assert "# Print final statistics" in head
    assert "# Send filter end event" in update
    
print("Edit extraction tests passed!")